# ver 1
<!-- Please modify the script to add two extra columns in the output table. The first column is "Status", and the second column is "Check Type".For "Status" column, fill the column with three options "OKAY", "BAD", and "CHECK".

If "PTM Type" and "Amino Acid and Site" each contains only one result, and the format of the result from "Amino Acid and Site" is "([A-a-z]+-[0-9]+)", and the PTM type correctly matches with the amino acid (
“Glycosylation Ser”
“Phosphorylation Ser”
“Phosphorylation Thr”
“Phosphorylation Tyr”
“Ubiquitination Lys”
“Acetylation Lys”
“Methylation Lys”
“Sumoylation Lys”
“Methylation Arg”
), mark the "Status" column as "OKAY" and leave "N/A" for "Check Type" column;

If the result from "Amino Acid and Site" is not "([A-a-z]+-[0-9]+)" OR the full name of an amino acid, mark the "Status" as "BAD" and leave "N/A" for "Check Type" column;

If 1) "PTM Type" has multiple results, OR 2) "Amino Acid and Site" has multiple results, OR 3) the "Amino Acid and Site" only has the full name of an amino acid without site information ("-[0-9]+"), OR 4) the PTM type incorrectly matches with the amino acid (refer to the list earlier), OR 5) all other rows that are not "OKAY" or "BAD", mark "Status" as "CHECK", and assign "CHECK TYPE" as "Multi Types", "Multi AA and Sites", "No Site", "Not Matching", and "Other" accordingly. -->


In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    status, check_type = determine_status(ptm_types, amino_acid_site, full_amino_acid)

    if ptm_types and (amino_acid_site or full_amino_acid):
        if not amino_acid_site:
            amino_acid_site = full_amino_acid
        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site,
            source,
            ptm_info,
            status,
            check_type
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]+-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid):
    ptm_type_list = ptm_types.split(', ')
    amino_acid_site_list = amino_acid_site.split(', ')

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"
    if len(amino_acid_site_list) > 1:
        return "CHECK", "Multi AA and Sites"
    if full_amino_acid and not amino_acid_site:
        return "CHECK", "No Site"

    if amino_acid_site:
        amino_acid, site = amino_acid_site.split('-')
        ptm_type = ptm_type_list[0]
        if not validate_ptm_type(ptm_type, amino_acid):
            return "CHECK", "Not Matching"
    else:
        return "BAD", "N/A"

    return "OKAY", "N/A"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': 'Ser',
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': 'Lys',
        'Acetylation': 'Lys',
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': 'Lys',
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            if isinstance(value, list) and amino_acid in value:
                return True
            elif amino_acid == value:
                return True
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Check Type'])
        writer.writerows(data)

def count_status(data):
    counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0}
    for row in data:
        status = row[6]  # "Status" column is at index 6
        if status in counts:
            counts[status] += 1
    return counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtration/ver1.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the statuses
status_counts = count_status(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 1026, 'BAD': 0, 'CHECK': 2330}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtration/ver1.csv


# ver 2
<!-- Modify the script to 1) add an additional condition for "CHECK": If "Source" is not PTM, mark "Status" as "CHECK", and assign "CHECK TYPE" as "Source". 2) capture rows where "Amino Acid and Site" is not "[A-a-z]+-[0-9]+ (one capital letter and two lower letters, three letters total with a dash and a number)" OR the full name of an amino acid, mark the "Status" as "BAD" and leave "N/A" for "Check Type" column. -->

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

    if ptm_types and (amino_acid_site or full_amino_acid):
        if not amino_acid_site:
            amino_acid_site = full_amino_acid
        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')
    amino_acid_site_list = amino_acid_site.split(', ')

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"
    if len(amino_acid_site_list) > 1:
        return "CHECK", "Multi AA and Sites"
    if full_amino_acid and not amino_acid_site:
        return "CHECK", "No Site"
    if source != "PTM":
        return "CHECK", "Source"
    if amino_acid_site and not re.match(r'[A-Za-z]{3}-[0-9]+', amino_acid_site):
        return "BAD", "N/A"
    if not (amino_acid_site or full_amino_acid):
        return "BAD", "N/A"

    if amino_acid_site:
        amino_acid, site = amino_acid_site.split('-')
        ptm_type = ptm_type_list[0]
        if not validate_ptm_type(ptm_type, amino_acid):
            return "CHECK", "Not Matching"

    return "OKAY", "N/A"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': 'Ser',
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': 'Lys',
        'Acetylation': 'Lys',
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': 'Lys',
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            if isinstance(value, list) and amino_acid in value:
                return True
            elif amino_acid == value:
                return True
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        writer.writerows(data)

def count_status(data):
    counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0}
    for row in data:
        status = row[6]  # "Status" column is at index 6
        if status in counts:
            counts[status] += 1
    return counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtration/ver2.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the statuses
status_counts = count_status(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 950, 'BAD': 0, 'CHECK': 2390}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtration/ver2.csv


# ver 3 - IGNORE added

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

    if ptm_types and (amino_acid_site or full_amino_acid):
        if not amino_acid_site:
            amino_acid_site = full_amino_acid
        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z][a-z]-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')
    amino_acid_site_list = amino_acid_site.split(', ')

    if full_amino_acid:
        return "IGNORE", "N/A"
    if 'Disulfide bond' in ptm_type_list:
        return "IGNORE", "N/A"
    if amino_acid_site and not all(re.match(r'[A-Z][a-z][a-z]-[0-9]+', site) for site in amino_acid_site_list):
        return "BAD", "N/A"

    if len(ptm_type_list) == 1 and amino_acid_site and all(re.match(r'[A-Z][a-z][a-z]-[0-9]+', site) for site in amino_acid_site_list):
        amino_acid = amino_acid_site_list[0].split('-')[0]
        if all(site.split('-')[0] == amino_acid for site in amino_acid_site_list):
            if validate_ptm_type(ptm_type_list[0], amino_acid):
                return "OKAY", "N/A"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"
    if len(amino_acid_site_list) > 1:
        return "CHECK", "Multi AA and Sites"
    if amino_acid_site:
        amino_acid, site = amino_acid_site.split('-')
        if not validate_ptm_type(ptm_type_list[0], amino_acid):
            return "CHECK", "Not Matching"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': 'Ser',
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': 'Lys',
        'Acetylation': 'Lys',
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': 'Lys',
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            if isinstance(value, list) and amino_acid in value:
                return True
            elif amino_acid == value:
                return True
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        writer.writerows(data)

def count_status(data):
    counts = {'OKAY': 0, 'BAD': 0, 'IGNORE': 0, 'CHECK': 0}
    for row in data:
        status = row[6]  # "Status" column is at index 6
        if status in counts:
            counts[status] += 1
    return counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver3.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the statuses
status_counts = count_status(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 1355, 'BAD': 0, 'IGNORE': 1077, 'CHECK': 822}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver3.csv


# ver 4

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1:
        aa_sites = amino_acid_site.split(', ')
        amino_acids = set(site.split('-')[0] for site in aa_sites)
        if len(amino_acids) == 1 and validate_ptm_type(ptm_type_list[0], list(amino_acids)[0]):
            return "OKAY", "N/A"

    # Condition 5: All other cases
    notation = "Other"
    if len(ptm_type_list) > 1:
        notation = "Multi Types"
    elif len(amino_acid_site.split(', ')) > 1:
        notation = "Multi AA and Sites"
    elif not validate_ptm_type(ptm_type_list[0], amino_acid_site.split('-')[0]):
        notation = "Not Matching"
    elif source != "PTM":
        notation = "Source"

    return "CHECK", notation

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys'],
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': ['Lys'],
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status(data):
    counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    for row in data:
        status = row[6]  # "Status" column is at index 6
        if status in counts:
            counts[status] += 1
    return counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver4.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the statuses
status_counts = count_status(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1320, 'BAD': 133, 'CHECK': 802, 'IGNORE': 1085}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver4.csv


# ver 5 - re-concatenate conditions

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')
    aa_sites = amino_acid_site.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            return "OKAY", "N/A"

    # Condition 5: All other cases
    if source != "PTM":
        return "CHECK", "Source"

    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys'],
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': ['Lys'],
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status(data):
    counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    for row in data:
        status = row[6]  # "Status" column is at index 6
        if status in counts:
            counts[status] += 1
    return counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver5.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the statuses
status_counts = count_status(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1320, 'BAD': 133, 'CHECK': 802, 'IGNORE': 1085}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver5.csv


# ver 6

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            return "OKAY", "N/A"

    # Condition 5: All other cases
    if source != "PTM":
        return "CHECK", "Source"

    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys'],
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': ['Lys'],
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status(data):
    counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    for row in data:
        status = row[6]  # "Status" column is at index 6
        if status in counts:
            counts[status] += 1
    return counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver7.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the statuses
status_counts = count_status(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1512, 'BAD': 206, 'CHECK': 911, 'IGNORE': 711}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver7.csv


# ver 7

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys'],
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': ['Lys'],
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver7.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1391, 'BAD': 126, 'CHECK': 1112, 'IGNORE': 711}
Notation Counts: {'Multi AA and Sites': 676, 'Multi Types': 347, 'N/A': 1787, 'Source': 257, 'Not Matching': 273}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver7.csv


# PTM list extraction

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
amino_acids = []
ptm_types = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        amino_acid = None
        ptm_type = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
        if ptm_type:  # Only keep rows with non-empty PTM Type
            identifiers.append(identifier)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types
})

# Save the DataFrame to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Table saved to {output_file_path}")


Table saved to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list.csv


# PTM list revised

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_2.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
feature_keys = []
amino_acids = []
ptm_types = []
positions_of_mod = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        feature_key = None
        amino_acid = None
        ptm_type = None
        position_of_mod = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('FT') and 'MOD_RES' in line:
                parts = line.split()
                feature_key = parts[1]
            elif line.startswith('PA'):
                position_of_mod = line[5:].strip()
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
        if ptm_type and feature_key == 'MOD_RES':  # Only keep rows with non-empty PTM Type and MOD_RES Feature Key
            identifiers.append(identifier)
            feature_keys.append(feature_key)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)
            positions_of_mod.append(position_of_mod)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Feature Key': feature_keys,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types,
    'Position of the Mod': positions_of_mod
})

# Save the DataFrame to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Table saved to {output_file_path}")


Table saved to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_2.csv


#PTM list revised 3

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_3.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
feature_keys = []
amino_acids = []
ptm_types = []
positions_of_mod = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        feature_key = None
        amino_acid = None
        ptm_type = None
        position_of_mod = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('FT') and 'MOD_RES' in line:
                parts = line.split()
                feature_key = parts[1]
            elif line.startswith('PA'):
                position_of_mod = line[5:].strip()
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
        # Only keep rows with non-empty PTM Type and MOD_RES Feature Key
        # Ignore specific PTM Types
        if ptm_type and feature_key == 'MOD_RES':
          if ptm_type not in ["D-amino acid.", "Thioether bond.", "D-amino acid; Thioether bond."]:
        # Add to lists
            identifiers.append(identifier)
            feature_keys.append(feature_key)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)
            positions_of_mod.append(position_of_mod)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Feature Key': feature_keys,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types,
    'Position of the Mod': positions_of_mod
})

# Save the DataFrame to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Table saved to {output_file_path}")


Table saved to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_3.csv


# ver 8
## - prioritized "BAD" before "Source"

In [ ]:
import csv
import re

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys'],
        'Methylation': ['Lys', 'Arg'],
        'Sumoylation': ['Lys'],
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver8.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1391, 'BAD': 206, 'CHECK': 1032, 'IGNORE': 711}
Notation Counts: {'Multi AA and Sites': 676, 'Multi Types': 347, 'N/A': 1867, 'Source': 177, 'Not Matching': 273}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver8.csv


# ver 9 (incomplete script dont know why)
## - flag dup

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat',
                's-nitrosylat', 'succinylat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[7] = "Duplicate"  # Mark notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver9.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1391, 'BAD': 206, 'CHECK': 1032, 'IGNORE': 711}
Notation Counts: {'Multi AA and Sites': 674, 'Multi Types': 333, 'N/A': 1842, 'Duplicate': 58, 'Source': 167, 'Not Matching': 266}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver9.csv


# PTM list ver 4
## -remove "position of mod"
## - add a new column with PSI-MOD
## - add LIPID and CARBOHYD to "Feature Key" column
## - change MOD:00482 to MOD:00030 (UniProt's PTM list is using a version of that modification that is considered an artifact (482) instead of the natural version (30).)

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_4.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
feature_keys = []
amino_acids = []
ptm_types = []
cross_references = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        feature_key = None
        amino_acid = None
        ptm_type = None
        cross_reference = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('FT') and any(ft_key in line for ft_key in ['MOD_RES', 'LIPID', 'CARBOHYD']):
                parts = line.split()
                feature_key = parts[1]
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
            elif line.startswith('DR') and 'PSI-MOD' in line:
                cross_reference = line[5:].strip()
                # Replace MOD:00482 with MOD:00030
                cross_reference = cross_reference.replace('MOD:00482', 'MOD:00030')
        # Only keep rows with non-empty PTM Type and Cross Reference and valid Feature Key
        if ptm_type and cross_reference and feature_key in ['MOD_RES', 'LIPID', 'CARBOHYD'] and ptm_type not in ["D-amino acid.", "Thioether bond.", "D-amino acid; Thioether bond."]:
            # Add to lists
            identifiers.append(identifier)
            feature_keys.append(feature_key)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)
            cross_references.append(cross_reference)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Feature Key': feature_keys,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types,
    'Cross Reference': cross_references
})

# Save the DataFrame to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Table saved to {output_file_path}")


Table saved to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_4.csv


# Ver 10
## - Add PTM ends with -ation from PTM list to the table
## - Add additional PTM&AA combinations from PTM list to the table


In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
        'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
        'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
        'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Za-z]{3}-[0-9]+)'
    matches = re.findall(pattern, text)
    return ', '.join(sorted(set(matches))) if matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not re.match(r'^([A-Z][a-z]{2}-\d+)(, [A-Z][a-z]{2}-\d+)*$', amino_acid_site):
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Glutathionylation': ['Cys'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
        'Methylation': ['Lys', 'Arg', 'Gln', 'Trp', 'Asp', 'Glu', 'Leu', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Thr', 'Phe'],
        'Sumoylation': ['Lys'],
        'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
        'Nitration': ['Tyr'],
        'Prenylation': ['Trp', 'Cys'],
        'Bromination': ['Trp', 'His'],
        'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
        'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
        'Citrullination': ['Arg'],
        'Oxidation': ['Cys', 'Met', 'Leu'],
        'Iodination': ['Tyr'],
        'Formylation': ['Gly', 'Met', 'Lys'],
        'S-nitrosylation': ['Cys'],
        'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
        'Glycation': ['Lys']
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[7] = "Duplicate"  # Mark notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver10.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1496, 'BAD': 268, 'CHECK': 1001, 'IGNORE': 769}
Notation Counts: {'Multi AA and Sites': 682, 'Multi Types': 400, 'N/A': 2055, 'Duplicate': 59, 'Source': 176, 'Not Matching': 162}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver10.csv


# PTM list ver 5
## -remove unecessary string "PSI-MOD;" from column E, only remaining MOD number
## -remove dups in column A&E
  ###-(empty dataframe)
  

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_5.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
feature_keys = []
amino_acids = []
ptm_types = []
cross_references = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        feature_key = None
        amino_acid = None
        ptm_type = None
        cross_reference = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('FT') and any(ft_key in line for ft_key in ['MOD_RES', 'LIPID', 'CARBOHYD']):
                parts = line.split()
                feature_key = parts[1]
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
            elif line.startswith('DR') and 'PSI-MOD' in line:
                cross_reference = line[14:].strip()
                # Replace MOD:00482 with MOD:00030
                cross_reference = cross_reference.replace('MOD:00482', 'MOD:00030')
        # Only keep rows with non-empty PTM Type and Cross Reference and valid Feature Key
        if ptm_type and cross_reference and feature_key in ['MOD_RES', 'LIPID', 'CARBOHYD'] and ptm_type not in ["D-amino acid.", "Thioether bond.", "D-amino acid; Thioether bond."]:
            # Add to lists
            identifiers.append(identifier)
            feature_keys.append(feature_key)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)
            cross_references.append(cross_reference)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Feature Key': feature_keys,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types,
    'Cross Reference': cross_references
})

# Remove the period at the end of strings in columns C, D, and E
df['Amino Acid'] = df['Amino Acid'].str.rstrip('.')
df['PTM Type'] = df['PTM Type'].str.rstrip('.')
df['Cross Reference'] = df['Cross Reference'].str.rstrip('.')

# Identify duplicates based on 'Identifier' and 'Cross Reference'
duplicates = df[df.duplicated(subset=['Identifier', 'Cross Reference'], keep=False)]

# Print the duplicates
print("Duplicates found:")
print(duplicates)

# Remove duplicates, keeping the first occurrence
df_cleaned = df.drop_duplicates(subset=['Identifier', 'Cross Reference'])

# Save the DataFrame to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Table saved to {output_file_path}")


Duplicates found:
Empty DataFrame
Columns: [Identifier, Feature Key, Amino Acid, PTM Type, Cross Reference]
Index: []
Table saved to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_5.csv


# PTM list - PTM and AA combination

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
feature_keys = []
amino_acids = []
ptm_types = []
cross_references = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        feature_key = None
        amino_acid = None
        ptm_type = None
        cross_reference = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('FT') and any(ft_key in line for ft_key in ['MOD_RES', 'LIPID', 'CARBOHYD']):
                parts = line.split()
                feature_key = parts[1]
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
            elif line.startswith('DR') and 'PSI-MOD' in line:
                cross_reference = line[14:].strip()
                # Replace MOD:00482 with MOD:00030
                cross_reference = cross_reference.replace('MOD:00482', 'MOD:00030')
        # Only keep rows with non-empty PTM Type and Cross Reference and valid Feature Key
        if ptm_type and cross_reference and feature_key in ['MOD_RES', 'LIPID', 'CARBOHYD'] and ptm_type not in ["D-amino acid.", "Thioether bond.", "D-amino acid; Thioether bond."]:
            # Add to lists
            identifiers.append(identifier)
            feature_keys.append(feature_key)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)
            cross_references.append(cross_reference)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Feature Key': feature_keys,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types,
    'Cross Reference': cross_references
})

# Remove the period at the end of strings in columns C, D, and E
df['Amino Acid'] = df['Amino Acid'].str.rstrip('.')
df['PTM Type'] = df['PTM Type'].str.rstrip('.')
df['Cross Reference'] = df['Cross Reference'].str.rstrip('.')

# Filter rows where 'PTM Type' ends with 'ation'
df_ation = df[df['PTM Type'].str.endswith('ation')]

# Identify duplicates based on 'Amino Acid' and 'PTM Type'
duplicates = df_ation[df_ation.duplicated(subset=['Amino Acid', 'PTM Type'], keep=False)]

# Print the duplicates
print("Duplicates found based on 'Amino Acid' and 'PTM Type':")
print(duplicates)

# Remove duplicates, keeping the first occurrence
df_ation_cleaned = df_ation.drop_duplicates(subset=['Amino Acid', 'PTM Type'])

# Save the cleaned DataFrame to a CSV file
df_ation_cleaned.to_csv(output_file_path, index=False)

print(f"Cleaned table saved to {output_file_path}")

Duplicates found based on 'Amino Acid' and 'PTM Type':
                           Identifier Feature Key  Amino Acid  \
0              (3R)-3-hydroxyarginine     MOD_RES    Arginine   
1            (3R)-3-hydroxyasparagine     MOD_RES  Asparagine   
2             (3R)-3-hydroxyaspartate     MOD_RES   Aspartate   
4        (3R,4R)-3,4-dihydroxyproline     MOD_RES     Proline   
5     (3R,4R)-4,5-dihydroxyisoleucine     MOD_RES  Isoleucine   
..                                ...         ...         ...   
283     O-linked (Ara) hydroxyproline    CARBOHYD     Proline   
290      O-linked (Gal) hydroxylysine    CARBOHYD      Lysine   
291     O-linked (Gal) hydroxyproline    CARBOHYD     Proline   
297      O-linked (Glc) hydroxylysine    CARBOHYD      Lysine   
301  O-linked (GlcNAc) hydroxyproline    CARBOHYD     Proline   

                        PTM Type Cross Reference  
0                  Hydroxylation       MOD:01956  
1                  Hydroxylation       MOD:00035  
2          

# VER 11
## -Remove "BAD" results

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
        'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
        'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
        'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
        'Glycosylation': ['Ser'],
        'Glutathionylation': ['Cys'],
        'Phosphorylation': ['Ser', 'Thr', 'Tyr'],
        'Ubiquitination': ['Lys'],
        'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
        'Methylation': ['Lys', 'Arg', 'Gln', 'Trp', 'Asp', 'Glu', 'Leu', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Thr', 'Phe'],
        'Sumoylation': ['Lys'],
        'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
        'Nitration': ['Tyr'],
        'Prenylation': ['Trp', 'Cys'],
        'Bromination': ['Trp', 'His'],
        'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
        'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
        'Citrullination': ['Arg'],
        'Oxidation': ['Cys', 'Met', 'Leu'],
        'Iodination': ['Tyr'],
        'Formylation': ['Gly', 'Met', 'Lys'],
        'S-nitrosylation': ['Cys'],
        'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
        'Glycation': ['Lys']
    }

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[7] = "Duplicate"  # Mark notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver11.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1536, 'BAD': 0, 'CHECK': 1035, 'IGNORE': 823}
Notation Counts: {'Multi AA and Sites': 704, 'Multi Types': 407, 'N/A': 1864, 'Duplicate': 63, 'Source': 192, 'Not Matching': 164}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver11.csv


# PTM list 6
## -convert 'Phosphoprotein' -> 'Phosphorylation'
## -convert 'Glycoprotein' -> 'Glycosylation'
## - convert 'Lipoprotein'-> 'Lipidation'

In [ ]:
import pandas as pd

# File path
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/ptmlist.txt'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_6.csv'

# Read the file and split entries by '//'
with open(input_file_path, 'r') as file:
    data = file.read().split('//\n')

# Initialize lists to store the table data
identifiers = []
feature_keys = []
amino_acids = []
ptm_types = []
cross_references = []

# Process each entry
for entry in data:
    if entry.strip():  # Skip empty entries
        lines = entry.strip().split('\n')
        identifier = None
        feature_key = None
        amino_acid = None
        ptm_type = None
        cross_reference = None
        for line in lines:
            if line.startswith('ID'):
                identifier = line[5:].strip()
            elif line.startswith('FT') and any(ft_key in line for ft_key in ['MOD_RES', 'LIPID', 'CARBOHYD']):
                parts = line.split()
                feature_key = parts[1]
            elif line.startswith('TG'):
                amino_acid = line[5:].strip()
            elif line.startswith('KW'):
                ptm_type = line[5:].strip()
            elif line.startswith('DR') and 'PSI-MOD' in line:
                cross_reference = line[14:].strip()
                # Replace MOD:00482 with MOD:00030
                cross_reference = cross_reference.replace('MOD:00482', 'MOD:00030')
        # Only keep rows with non-empty PTM Type and Cross Reference and valid Feature Key
        if ptm_type and cross_reference and feature_key in ['MOD_RES', 'LIPID', 'CARBOHYD'] and ptm_type not in ["D-amino acid.", "Thioether bond.", "D-amino acid; Thioether bond."]:
            # Add to lists
            identifiers.append(identifier)
            feature_keys.append(feature_key)
            amino_acids.append(amino_acid)
            ptm_types.append(ptm_type)
            cross_references.append(cross_reference)

# Create a DataFrame
df = pd.DataFrame({
    'Identifier': identifiers,
    'Feature Key': feature_keys,
    'Amino Acid': amino_acids,
    'PTM Type': ptm_types,
    'Cross Reference': cross_references
})

# Remove the period at the end of strings in columns C, D, and E
df['Amino Acid'] = df['Amino Acid'].str.rstrip('.')
df['PTM Type'] = df['PTM Type'].str.rstrip('.')
df['Cross Reference'] = df['Cross Reference'].str.rstrip('.')

# Convert "Phosphoprotein" to "Phosphorylation" and "Glycoprotein" to "Glycosylation"
def convert_ptm_type(ptm_string):
    conversions = {
        'Phosphoprotein': 'Phosphorylation',
        'Glycoprotein': 'Glycosylation',
        'Lipoprotein': 'Lipidation'
    }
    ptm_list = [ptm.strip() for ptm in ptm_string.split(';')]
    converted_list = [conversions.get(ptm, ptm) for ptm in ptm_list]
    return '; '.join(converted_list)

# Apply the conversion function to the 'PTM Type' column
df['PTM Type'] = df['PTM Type'].apply(convert_ptm_type)

# Identify duplicates based on 'Identifier' and 'Cross Reference'
duplicates = df[df.duplicated(subset=['Identifier', 'Cross Reference'], keep=False)]

# Print the duplicates
print("Duplicates found:")
print(duplicates)

# Remove duplicates, keeping the first occurrence
df_cleaned = df.drop_duplicates(subset=['Identifier', 'Cross Reference'])

# Save the DataFrame to a CSV file
df_cleaned.to_csv(output_file_path, index=False)
print(f"Table saved to {output_file_path}")


Duplicates found:
Empty DataFrame
Columns: [Identifier, Feature Key, Amino Acid, PTM Type, Cross Reference]
Index: []
Table saved to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_6.csv


# PTM and AA combination_2

In [ ]:
import pandas as pd

# File paths
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_6.csv'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination_2.csv'

# Read the CSV file generated from the previous script
df = pd.read_csv(input_file_path)

# Function to check if any part of the PTM Type ends with 'ation'
def ends_with_ation(ptm_type):
    return any(part.strip().endswith('ation') for part in ptm_type.split(';'))

# Filter rows where any part of 'PTM Type' ends with 'ation'
df_ation = df[df['PTM Type'].apply(ends_with_ation)]

# Identify duplicates based on 'Amino Acid' and 'PTM Type'
duplicates = df_ation[df_ation.duplicated(subset=['Amino Acid', 'PTM Type'], keep=False)]

# Print the duplicates
print("Duplicates found based on 'Amino Acid' and 'PTM Type':")
print(duplicates)

# Remove duplicates, keeping the first occurrence
df_ation_cleaned = df_ation.drop_duplicates(subset=['Amino Acid', 'PTM Type'])

# Save the cleaned DataFrame to a CSV file
df_ation_cleaned.to_csv(output_file_path, index=False)

print(f"Cleaned table saved to {output_file_path}")

Duplicates found based on 'Amino Acid' and 'PTM Type':
                          Identifier Feature Key  Amino Acid  \
0             (3R)-3-hydroxyarginine     MOD_RES    Arginine   
1           (3R)-3-hydroxyasparagine     MOD_RES  Asparagine   
2            (3R)-3-hydroxyaspartate     MOD_RES   Aspartate   
4       (3R,4R)-3,4-dihydroxyproline     MOD_RES     Proline   
5    (3R,4R)-4,5-dihydroxyisoleucine     MOD_RES  Isoleucine   
..                               ...         ...         ...   
307          O-linked (Man1P) serine    CARBOHYD      Serine   
309            O-linked (Xyl) serine    CARBOHYD      Serine   
314          S-linked (Gal) cysteine    CARBOHYD    Cysteine   
315          S-linked (Glc) cysteine    CARBOHYD    Cysteine   
316       S-linked (GlcNAc) cysteine    CARBOHYD    Cysteine   

                           PTM Type Cross Reference  
0                     Hydroxylation       MOD:01956  
1                     Hydroxylation       MOD:00035  
2             

# ver 12
## - added more combinations from "phosphorylation", "glycosylation", and "lipidation"
## -marked "Dup" in "Notation"

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
        'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
        'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
        'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[7] = "Duplicate"  # Mark notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver12.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1658, 'BAD': 0, 'CHECK': 913, 'IGNORE': 823}
Notation Counts: {'Multi AA and Sites': 752, 'Multi Types': 407, 'N/A': 1950, 'Duplicate': 63, 'Source': 192, 'Not Matching': 30}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver12.csv


## -marked "Dup" in "Notation"

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
        'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
        'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
        'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver12.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1646, 'BAD': 0, 'CHECK': 887, 'IGNORE': 861}
Notation Counts: {'Multi AA and Sites': 752, 'Multi Types': 407, 'N/A': 1950, 'Duplicate': 63, 'Source': 192, 'Not Matching': 30}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver12.csv


# ver 13
## -Map PTM type & AA in filtering list to unique MOD id in PTM combination list


In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
        'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
        'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
        'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "OKAY", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

def load_ptm_combinations(file_path):
    ptm_combinations = {}
    amino_acid_map = {
        'Alanine': 'Ala', 'Arginine': 'Arg', 'Asparagine': 'Asn', 'Aspartic acid': 'Asp',
        'Cysteine': 'Cys', 'Glutamic acid': 'Glu', 'Glutamine': 'Gln', 'Glycine': 'Gly',
        'Histidine': 'His', 'Isoleucine': 'Ile', 'Leucine': 'Leu', 'Lysine': 'Lys',
        'Methionine': 'Met', 'Phenylalanine': 'Phe', 'Proline': 'Pro', 'Serine': 'Ser',
        'Threonine': 'Thr', 'Tryptophan': 'Trp', 'Tyrosine': 'Tyr', 'Valine': 'Val'
    }

    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            amino_acid = amino_acid_map.get(row['Amino Acid'], row['Amino Acid'])
            key = (amino_acid, row['PTM Type'])
            ptm_combinations[key] = row['Cross Reference']

    return ptm_combinations

def map_cross_reference(ptm_data, ptm_combinations):
    for row in ptm_data:
        ptm_type = row[2]
        amino_acid_sites = row[3].split(', ')
        cross_references = set()

        for site in amino_acid_sites:
            amino_acid = site.split('-')[0]
            key = (amino_acid, ptm_type)
            if key in ptm_combinations:
                cross_references.add(ptm_combinations[key])

        row.append(', '.join(sorted(cross_references)) if cross_references else 'N/A')

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_2_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver13.csv'
ptm_combinations_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination_2.csv'

# Load PTM combinations
ptm_combinations = load_ptm_combinations(ptm_combinations_file)

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Map Cross References
map_cross_reference(deduplicated_ptm_data, ptm_combinations)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation', 'Cross Reference'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1646, 'BAD': 0, 'CHECK': 887, 'IGNORE': 861}
Notation Counts: {'Multi AA and Sites': 752, 'Multi Types': 407, 'N/A': 1950, 'Duplicate': 63, 'Source': 192, 'Not Matching': 30}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver13.csv


# PTM and AA combination_3
## - keep all unique MOD ID (keep column C&D dups)

In [ ]:
import pandas as pd

# File paths
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_6.csv'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination_3.csv'

# Read the CSV file generated from the previous script
df = pd.read_csv(input_file_path)

# Function to check if any part of the PTM Type ends with 'ation'
def ends_with_ation(ptm_type):
    return any(part.strip().endswith('ation') for part in ptm_type.split(';'))

# Filter rows where any part of 'PTM Type' ends with 'ation'
df_ation = df[df['PTM Type'].apply(ends_with_ation)]

# Find duplicates in column E ('Cross Reference')
duplicates = df_ation['Cross Reference'].duplicated(keep=False)

# Print duplicates
print("Duplicates in 'Cross Reference' column:")
print(df_ation[duplicates]['Cross Reference'])

# # Save the cleaned DataFrame to a CSV file
# df_ation.to_csv(output_file_path, index=False)

# print(f"Table saved to {output_file_path}")

Duplicates in 'Cross Reference' column:
53     MOD:00177
54     MOD:00177
281    MOD:00165
282    MOD:00165
Name: Cross Reference, dtype: object


# Remove all lines that are not in "ID" "AC" "CC" and "FT"

In [ ]:
# Read the file content
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/UP000005640_9606.dat copy'

# Function to remove lines that do not start with "ID", "AC", or "CC"
def filter_uniprot_file(file_path):
    filtered_lines = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("ID") or line.startswith("AC") or line.startswith("CC") or line.startswith("FT") or line.startswith("//"):
                filtered_lines.append(line)
    return filtered_lines

# Filter the file
filtered_lines = filter_uniprot_file(file_path)

# Save the filtered content to a new file
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_3_UP000005640_9606.dat'
with open(output_file_path, 'w') as file:
    file.writelines(filtered_lines)

output_file_path


'/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_3_UP000005640_9606.dat'

## -removed " ---------------------------------------------------------------------------
# CC   Copyrighted by the UniProt Consortium, see https://www.uniprot.org/terms
# CC   Distributed under the Creative Commons Attribution (CC BY 4.0) License
# CC   ---------------------------------------------------------------------------" from the file

In [ ]:
# Function to filter UniProt file and keep specific FT lines with two lines below
def filter_uniprot_file(file_path):
    filtered_lines = []
    copyright_section = [
        "CC   ---------------------------------------------------------------------------\n",
        "CC   Copyrighted by the UniProt Consortium, see https://www.uniprot.org/terms\n",
        "CC   Distributed under the Creative Commons Attribution (CC BY 4.0) License\n",
        "CC   ---------------------------------------------------------------------------\n"
    ]
    with open(file_path, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i]

            # Skip the copyright section
            if line in copyright_section:
                i += 1
                continue

            # Keep lines starting with "ID", "AC", "CC", or "//"
            if line.startswith(("ID", "AC", "CC", "//")):
                filtered_lines.append(line)
                i += 1
                continue

            # Check if the line starts with "FT MOD_RES", "FT CARBOHYD", or "FT LIPID"
            if line.startswith(("FT   MOD_RES", "FT   CARBOHYD", "FT   LIPID")):
                # Append this line and the next two lines
                filtered_lines.append(line)
                if i + 1 < len(lines):
                    filtered_lines.append(lines[i + 1])
                if i + 2 < len(lines):
                    filtered_lines.append(lines[i + 2])
                # Skip the next two lines (as we've already processed them)
                i += 3
            else:
                # Skip all other FT lines that are not MOD_RES, CARBOHYD, or LIPID
                i += 1

    return filtered_lines

# File paths
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/UP000005640_9606.dat copy'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_5_UP000005640_9606.dat'

# Filter the file
filtered_lines = filter_uniprot_file(input_file_path)

# Save the filtered content to a new file
with open(output_file_path, 'w') as file:
    file.writelines(filtered_lines)

print(f"Filtered content saved to {output_file_path}")


Filtered content saved to /content/drive/MyDrive/Protein Ontology-text mining/data/filtered_5_UP000005640_9606.dat


# ver 14
## - re-mapped combination list to extraction list (kept unique MOD IDs and specific PTM from 1st column)
#### - removed MOD:00177 and MOD:00165 (dups) from PTM combination list
## - marked all "Multi AA and Sites" and "Multi Types" as "CHECK"

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        lines = [line for line in lines if line.startswith('ID') or line.startswith('CC') or line.startswith('AC')]

        if not lines:
            continue

        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM')
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
        'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
        'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
        'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in lines[lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    # Only process if there's content in both ptm_types and (amino_acid_site or full_amino_acid)
    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation
        ])

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "CHECK", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

def load_ptm_combinations(file_path):
    ptm_combinations = {}
    amino_acid_map = {
        'Alanine': 'Ala', 'Arginine': 'Arg', 'Asparagine': 'Asn', 'Aspartic acid': 'Asp',
        'Cysteine': 'Cys', 'Glutamic acid': 'Glu', 'Glutamine': 'Gln', 'Glycine': 'Gly',
        'Histidine': 'His', 'Isoleucine': 'Ile', 'Leucine': 'Leu', 'Lysine': 'Lys',
        'Methionine': 'Met', 'Phenylalanine': 'Phe', 'Proline': 'Pro', 'Serine': 'Ser',
        'Threonine': 'Thr', 'Tryptophan': 'Trp', 'Tyrosine': 'Tyr', 'Valine': 'Val'
    }

    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            amino_acid = amino_acid_map.get(row['Amino Acid'], row['Amino Acid'])
            key = (amino_acid, row['PTM Type'])
            ptm_combinations[key] = row['Cross Reference']

    return ptm_combinations

def map_cross_reference(ptm_data, ptm_combinations):
    for row in ptm_data:
        ptm_type = row[2]
        amino_acid_sites = row[3].split(', ')
        cross_references = set()

        for site in amino_acid_sites:
            amino_acid = site.split('-')[0]
            key = (amino_acid, ptm_type)
            if key in ptm_combinations:
                cross_references.add(ptm_combinations[key])

        row.append(', '.join(sorted(cross_references)) if cross_references else 'N/A')

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_3_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver14.csv'
ptm_combinations_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination_3.csv'

# Load PTM combinations
ptm_combinations = load_ptm_combinations(ptm_combinations_file)

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Map Cross References
map_cross_reference(deduplicated_ptm_data, ptm_combinations)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation', 'Cross Reference'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")

Status Counts: {'OKAY': 1152, 'BAD': 0, 'CHECK': 1381, 'IGNORE': 861}
Notation Counts: {'Multi AA and Sites': 752, 'Multi Types': 407, 'N/A': 1950, 'Duplicate': 63, 'Source': 192, 'Not Matching': 30}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver14.csv


# Keep FT lines with "FT MOD_RES", "FT CARBOHYD", or "FT LIPID" only in the input file

In [ ]:
# Function to filter UniProt file and keep specific FT lines with two lines below
def filter_uniprot_file(file_path):
    filtered_lines = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i]

            # Keep lines starting with "ID", "AC", "CC", or "//"
            if line.startswith(("ID", "AC", "CC", "//")):
                filtered_lines.append(line)
                i += 1
                continue

            # Check if the line starts with "FT MOD_RES", "FT CARBOHYD", or "FT LIPID"
            if line.startswith(("FT   MOD_RES", "FT   CARBOHYD", "FT   LIPID")):
                # Append this line and the next two lines
                filtered_lines.append(line)
                if i + 1 < len(lines):
                    filtered_lines.append(lines[i + 1])
                if i + 2 < len(lines):
                    filtered_lines.append(lines[i + 2])
                # Skip the next two lines (as we've already processed them)
                i += 3
            else:
                # Skip all other FT lines that are not MOD_RES, CARBOHYD, or LIPID
                i += 1

    return filtered_lines

# File paths
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/UP000005640_9606.dat copy'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_4_UP000005640_9606.dat'

# Filter the file
filtered_lines = filter_uniprot_file(input_file_path)

# Save the filtered content to a new file
with open(output_file_path, 'w') as file:
    file.writelines(filtered_lines)

print(f"Filtered content saved to {output_file_path}")

Filtered content saved to /content/drive/MyDrive/Protein Ontology-text mining/data/filtered_4_UP000005640_9606.dat


# Remove URL link lines from input file

In [ ]:
def filter_uniprot_file(file_path):
    filtered_lines = []
    copyright_section = [
        "CC   ---------------------------------------------------------------------------\n",
        "CC   Copyrighted by the UniProt Consortium, see https://www.uniprot.org/terms\n",
        "CC   Distributed under the Creative Commons Attribution (CC BY 4.0) License\n",
        "CC   ---------------------------------------------------------------------------\n"
    ]
    with open(file_path, 'r') as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i]

            # Skip the copyright section
            if line in copyright_section:
                i += 1
                continue

            # Skip lines starting with "CC       URL="
            if line.startswith("CC       URL="):
                i += 1
                continue

            # Keep lines starting with "ID", "AC", "CC", or "//"
            if line.startswith(("ID", "AC", "CC", "//")):
                filtered_lines.append(line)
                i += 1
                continue

            # Check if the line starts with "FT MOD_RES", "FT CARBOHYD", or "FT LIPID"
            if line.startswith(("FT   MOD_RES", "FT   CARBOHYD", "FT   LIPID")):
                # Append this line and the next two lines
                filtered_lines.append(line)
                if i + 1 < len(lines):
                    filtered_lines.append(lines[i + 1])
                if i + 2 < len(lines):
                    filtered_lines.append(lines[i + 2])
                # Skip the next two lines (as we've already processed them)
                i += 3
            else:
                # Skip all other FT lines that are not MOD_RES, CARBOHYD, or LIPID
                i += 1

    return filtered_lines

# File paths
input_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/UP000005640_9606.dat copy'
output_file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_6_UP000005640_9606.dat'

# Filter the file
filtered_lines = filter_uniprot_file(input_file_path)

# Save the filtered content to a new file
with open(output_file_path, 'w') as file:
    file.writelines(filtered_lines)

print(f"Filtered content saved to {output_file_path}")

Filtered content saved to /content/drive/MyDrive/Protein Ontology-text mining/data/filtered_6_UP000005640_9606.dat


# VER 15
## -removed cross-ref mapping
## -added identifiers from input file


In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        ft_lines = [line for line in lines if line.startswith('FT')]
        cc_lines = [line for line in lines if line.startswith('CC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in cc_lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines)
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines)
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
                'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
                'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in cc_lines[cc_lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines=ft_lines)

        if capturing:
            process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines)

    return ptm_data

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    if ptm_types and (amino_acid_site or full_amino_acid):
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source)

        identifier = extract_identifier(amino_acid_site, ft_lines) if status in ("CHECK", "OKAY") else "N/A"

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation,
            identifier
        ])

def extract_identifier(amino_acid_site, ft_lines):
    if not amino_acid_site:
        return "N/A"

    site_numbers = re.findall(r'\d+', amino_acid_site)
    identifiers = []

    for number in site_numbers:
        identifier = "N/A"
        for i, line in enumerate(ft_lines):
            if line.startswith(('FT   MOD_RES', 'FT   CARBOHYD', 'FT   LIPID')):  # Match with leading spaces
                line_parts = line.split()
                if len(line_parts) > 2 and line_parts[2] == number:
                    note_match = line[28:].strip().strip('"')
                    if note_match:
                        identifier = note_match.group(1)
                    else:
                        if i + 1 < len(ft_lines):
                            next_line = ft_lines[i + 1].strip()  # Strip leading spaces from the next line
                            if '/note=' in next_line:
                                note_match = re.search(r'/note="([^"]+)"', next_line)
                                if note_match:
                                    identifier = note_match.group(1)
                    break
        identifiers.append(identifier)

    return ", ".join(identifiers)

def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source):
    ptm_type_list = ptm_types.split(', ')

    # Condition 1: Full name of amino acid(s)
    if full_amino_acid and not amino_acid_site:
        return "IGNORE", "N/A"

    # Condition 2: Disulfide bond
    if "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 3: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    aa_sites = amino_acid_site.split(', ')

    # First, check the source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 4: Single PTM type, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(set(site.split('-')[0] for site in aa_sites)) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            if len(aa_sites) > 1:
                return "CHECK", "Multi AA and Sites"
            else:
                return "OKAY", "N/A"

    # Condition 5: All other cases
    if len(ptm_type_list) == 1 and not validate_ptm_type(ptm_type_list[0], aa_sites[0].split('-')[0]):
        return "CHECK", "Not Matching"

    if len(ptm_type_list) > 1:
        return "CHECK", "Multi Types"

    if len(aa_sites) > 1:
        return "CHECK", "Multi AA and Sites"

    return "CHECK", "Other"

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation', 'Identifier'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_6_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver15.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 1152, 'BAD': 0, 'CHECK': 1381, 'IGNORE': 861}
Notation Counts: {'Multi AA and Sites': 752, 'Multi Types': 407, 'N/A': 1950, 'Duplicate': 63, 'Source': 192, 'Not Matching': 30}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver15.csv


# ver 16
## - more explicit Notion column: "1 PTM and Multi Sites" and “Multi PTM and Multi Sites”
## - added "MANUAL": "Uncertainty", "neighboring sites", and “Multi PTM and 1 Sites”
## - added "SKIP": "Not Matching": CHECK -> SKIP & "Alternate"
## - Used different delimiter ( &&& ) to separate values in identifier column


In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        ft_lines = [line for line in lines if line.startswith('FT')]
        cc_lines = [line for line in lines if line.startswith('CC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in cc_lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines)
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines)
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
                'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
                'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in cc_lines[cc_lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines=ft_lines)

        if capturing:
            process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines)

    return ptm_data


def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source, identifier, full_description):
    ptm_type_list = ptm_types.split(', ')
    aa_sites = amino_acid_site.split(', ') if amino_acid_site else []

    # Condition 1: IGNORE cases
    if (full_amino_acid and not amino_acid_site) or "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 2: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    # Condition 3: SKIP - Not matching PTM type and amino acid
    if len(ptm_type_list) == 1 and len(aa_sites) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if not validate_ptm_type(ptm_type_list[0], amino_acid):
            return "SKIP", "Not Matching"

    # Condition 4: Check source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 5: Single PTM type, single amino acid site, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(aa_sites) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            return "OKAY", "N/A"

    # Condition 6: SKIP - Alternate
    if 'alternate' in identifier.lower():
        return "SKIP", "Alternate"

    # Condition 7: MANUAL cases
    uncertainty_keywords = ["uncertain", "unknown", "unclear", "unsure", "not certain", "not known", "not clear", "not sure", "ambiguous", "undetermined"]
    if any(keyword in full_description.lower() for keyword in uncertainty_keywords):
        return "MANUAL", "Uncertainty"

    if len(aa_sites) > 1:
        site_numbers = [int(site.split('-')[1]) for site in aa_sites]
        site_numbers.sort()
        for i in range(len(site_numbers) - 1):
            if abs(site_numbers[i + 1] - site_numbers[i]) == 1:
                return "MANUAL", "Neighboring Sites"

    if len(ptm_type_list) > 1 and len(aa_sites) == 1:
        return "MANUAL", "Multi PTM and 1 Site"

    # Condition 8: CHECK - Multiple amino acid sites
    if len(aa_sites) > 1:
        if len(ptm_type_list) == 1:
            return "CHECK", "1 PTM and Multi Sites"
        else:
            return "CHECK", "Multi PTM and Multi Sites"

    # Default case
    return "CHECK", "Other"

def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    if ptm_types and (amino_acid_site or full_amino_acid):
        identifier = extract_identifier(amino_acid_site, ft_lines) if amino_acid_site else "N/A"
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source, identifier, ptm_info)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation,
            identifier
        ])

def extract_identifier(amino_acid_site, ft_lines):
    if not amino_acid_site:
        return "N/A"

    site_numbers = re.findall(r'\d+', amino_acid_site)
    identifiers = []

    for number in site_numbers:
        identifier = "N/A"
        for i, line in enumerate(ft_lines):
            if line.startswith(('FT   MOD_RES', 'FT   CARBOHYD', 'FT   LIPID')):  # Match with leading spaces
                line_parts = line.split()
                if len(line_parts) > 2 and line_parts[2] == number:
                    note_match = line[28:].strip().strip('"')
                    if note_match:
                        identifier = note_match.group(1)
                    else:
                        if i + 1 < len(ft_lines):
                            next_line = ft_lines[i + 1].strip()  # Strip leading spaces from the next line
                            if '/note=' in next_line:
                                note_match = re.search(r'/note="([^"]+)"', next_line)
                                if note_match:
                                    identifier = note_match.group(1)
                    break
        identifiers.append(identifier)

    return " &&& ".join(identifiers)

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation', 'Identifier'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0, 'SKIP': 0, 'MANUAL': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_6_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver16.csv'

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 1152, 'BAD': 0, 'CHECK': 964, 'IGNORE': 861, 'SKIP': 179, 'MANUAL': 238}
Notation Counts: {'Alternate': 161, 'N/A': 1950, 'Duplicate': 63, 'Neighboring Sites': 105, 'Source': 191, 'Multi PTM and 1 Site': 121, '1 PTM and Multi Sites': 619, 'Multi PTM and Multi Sites': 154, 'Not Matching': 18, 'Uncertainty': 12}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver16.csv


In [ ]:
# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_6_UP000005640_9606.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver16.csv'

# ver 17
## - MOD ID mapping

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path, mod_mappings):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        ft_lines = [line for line in lines if line.startswith('FT')]
        cc_lines = [line for line in lines if line.startswith('CC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in cc_lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines, mod_mappings=mod_mappings)
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines, mod_mappings=mod_mappings)
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
                'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
                'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in cc_lines[cc_lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines=ft_lines, mod_mappings=mod_mappings)

        if capturing:
            process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines, mod_mappings=mod_mappings)

    return ptm_data


def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source, identifier, full_description):
    ptm_type_list = ptm_types.split(', ')
    aa_sites = amino_acid_site.split(', ') if amino_acid_site else []

    # Condition 1: IGNORE cases
    if (full_amino_acid and not amino_acid_site) or "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 2: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    # Condition 3: SKIP - Not matching PTM type and amino acid
    if len(ptm_type_list) == 1 and len(aa_sites) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if not validate_ptm_type(ptm_type_list[0], amino_acid):
            return "SKIP", "Not Matching"

    # Condition 4: Check source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 5: Single PTM type, single amino acid site, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(aa_sites) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            return "OKAY", "N/A"

    # Condition 6: SKIP - Alternate
    if 'alternate' in identifier.lower():
        return "SKIP", "Alternate"

    # Condition 7: MANUAL cases
    uncertainty_keywords = ["uncertain", "unknown", "unclear", "unsure", "not certain", "not known", "not clear", "not sure", "ambiguous", "undetermined"]
    if any(keyword in full_description.lower() for keyword in uncertainty_keywords):
        return "MANUAL", "Uncertainty"

    if len(aa_sites) > 1:
        site_numbers = [int(site.split('-')[1]) for site in aa_sites]
        site_numbers.sort()
        for i in range(len(site_numbers) - 1):
            if abs(site_numbers[i + 1] - site_numbers[i]) == 1:
                return "MANUAL", "Neighboring Sites"

    if len(ptm_type_list) > 1 and len(aa_sites) == 1:
        return "MANUAL", "Multi PTM and 1 Site"

    # Condition 8: CHECK - Multiple amino acid sites
    if len(aa_sites) > 1:
        if len(ptm_type_list) == 1:
            return "CHECK", "1 PTM and Multi Sites"
        else:
            return "CHECK", "Multi PTM and Multi Sites"

    # Default case
    return "CHECK", "Other"

def load_mod_mappings(file_path):
    mod_mappings = {}
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            identifier, feature_key, amino_acid, ptm_type, mod_id = row
            mod_mappings[identifier.split(';')[0].strip()] = mod_id
    return mod_mappings

def map_identifiers_to_mod(identifiers, mod_mappings):
    if identifiers == "N/A":
        return "N/A"

    mapped_mods = []
    for identifier in identifiers.split(" &&& "):
        base_identifier = identifier.split(';')[0].strip()
        if base_identifier in mod_mappings:
            mapped_mods.append(mod_mappings[base_identifier])
        else:
            mapped_mods.append("Not Found")

    return " &&& ".join(mapped_mods)


def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines, mod_mappings):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    if ptm_types and (amino_acid_site or full_amino_acid):
        identifier = extract_identifier(amino_acid_site, ft_lines) if amino_acid_site else "N/A"
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source, identifier, ptm_info)
        cross_reference = map_identifiers_to_mod(identifier, mod_mappings)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation,
            identifier,
            cross_reference
        ])

def extract_identifier(amino_acid_site, ft_lines):
    if not amino_acid_site:
        return "N/A"

    site_numbers = re.findall(r'\d+', amino_acid_site)
    identifiers = []

    for number in site_numbers:
        identifier = "N/A"
        for i, line in enumerate(ft_lines):
            if line.startswith(('FT   MOD_RES', 'FT   CARBOHYD', 'FT   LIPID')):  # Match with leading spaces
                line_parts = line.split()
                if len(line_parts) > 2 and line_parts[2] == number:
                    note_match = line[28:].strip().strip('"')
                    if note_match:
                        identifier = note_match.group(1)
                    else:
                        if i + 1 < len(ft_lines):
                            next_line = ft_lines[i + 1].strip()  # Strip leading spaces from the next line
                            if '/note=' in next_line:
                                note_match = re.search(r'/note="([^"]+)"', next_line)
                                if note_match:
                                    identifier = note_match.group(1)
                    break
        identifiers.append(identifier)

    return " &&& ".join(identifiers)

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation', 'Identifier', 'Cross Reference'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0, 'SKIP': 0, 'MANUAL': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path_1 = '/content/drive/MyDrive/Protein Ontology-text mining/data/filtered_6_UP000005640_9606.dat'
file_path_2 = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination_3.csv'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver17.csv'

# Load MOD mappings from input file 2
mod_mappings = load_mod_mappings(file_path_2)

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path_1, mod_mappings)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 1152, 'BAD': 0, 'CHECK': 964, 'IGNORE': 861, 'SKIP': 179, 'MANUAL': 238}
Notation Counts: {'Alternate': 161, 'N/A': 1950, 'Duplicate': 63, 'Neighboring Sites': 105, 'Source': 191, 'Multi PTM and 1 Site': 121, '1 PTM and Multi Sites': 619, 'Multi PTM and Multi Sites': 154, 'Not Matching': 18, 'Uncertainty': 12}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver17.csv


# test on data subset

In [ ]:
import csv
import re
from collections import defaultdict

def parse_uniprot_file(file_path, mod_mappings):
    with open(file_path, 'r') as file:
        content = file.read()

    entries = content.split('//')
    ptm_data = []

    for entry in entries:
        lines = entry.strip().split('\n')
        id_line = next((line for line in lines if line.startswith('ID')), None)
        ac_lines = [line for line in lines if line.startswith('AC')]
        ft_lines = [line for line in lines if line.startswith('FT')]
        cc_lines = [line for line in lines if line.startswith('CC')]
        if not id_line or not ac_lines:
            continue

        entry_name = id_line.split()[1]
        first_accession = ac_lines[0].split()[1].strip(';')

        ptm_info = ''
        capturing = False

        for line in cc_lines:
            if line.startswith('CC   -!- PTM:'):
                if capturing:
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines, mod_mappings=mod_mappings)
                capturing = True
                ptm_info = line[13:].strip()
            elif capturing and line.startswith('CC       '):
                ptm_info += ' ' + line[8:].strip()
            elif capturing and (line.startswith('CC   -!-') or line.startswith('CC   -----')):
                capturing = False
                process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines, mod_mappings=mod_mappings)
                ptm_info = ''

            if not capturing and any(keyword in line.lower() for keyword in [
                'ubiquitinat', 'phosphorylat', 'acetylat', 'methylat', 'glycosylat', 'sumoylat',
                'disulfide bond', 'glycat', 'glutathionylat', 'hydroxylat', 'lipidat', 's-nitrosylat',
                'succinylat', 'prenylat', 'nitrate', 'brominat', 'adp-ribosylat', 'amidat', 'citrullinat',
                'oxidat', 'iodinat', 'formylat', 'sulfat']):
                source_match = re.search(r'CC\s+-!- (\w+):', line)
                if source_match:
                    source = source_match.group(1)
                    ptm_info = line[4:].strip()
                    for l in cc_lines[cc_lines.index(line)+1:]:
                        if l.startswith('CC   -!-') or l.startswith('CC   -----'):
                            break
                        ptm_info += ' ' + l[8:].strip()
                    process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines=ft_lines, mod_mappings=mod_mappings)

        if capturing:
            process_ptm(first_accession, entry_name, ptm_info, ptm_data, source='PTM', ft_lines=ft_lines, mod_mappings=mod_mappings)

    return ptm_data


def extract_all_ptm_types(text):
    ptm_types = {
        'Ubiquitination': ['ubiquitinat', 'ubiquitylat'],
        'Phosphorylation': ['phosphorylat'],
        'Acetylation': ['acetylat'],
        'Methylation': ['methylat'],
        'Glycosylation': ['glycosylat'],
        'Sumoylation': ['sumoylat'],
        'Disulfide bond': ['disulfide bond'],
        'Glycation': ['glycat'],
        'Glutathionylation': ['glutathionylat'],
        'Hydroxylation': ['hydroxylat'],
        'Lipidation': ['lipidat'],
        'S-nitrosylation': ['s-nitrosylat'],
        'Succinylation': ['succinylat'],
        'Prenylation': ['prenylat'],
        'Nitration': ['nitrate'],
        'Bromination': ['brominat'],
        'ADP-ribosylation': ['adp-ribosylat'],
        'Amidation': ['amidat'],
        'Citrullination': ['citrullinat'],
        'Oxidation': ['oxidat'],
        'Iodination': ['iodinat'],
        'Formylation': ['formylat'],
        'Sulfation': ['sulfat'],
    }

    found_ptms = []
    for ptm, keywords in ptm_types.items():
        if any(keyword in text.lower() for keyword in keywords):
            found_ptms.append(ptm)
    return ', '.join(found_ptms)

def extract_amino_acid_site(text):
    pattern = r'([A-Z][a-z]{2}-\d+)'
    matches = re.findall(pattern, text)
    valid_aa_codes = set([
        'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile',
        'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val'
    ])
    valid_matches = [match for match in matches if match.split('-')[0] in valid_aa_codes]
    return ', '.join(sorted(set(valid_matches))) if valid_matches else ''

def extract_amino_acid_fullname(text):
    amino_acids = [
        'alanine', 'arginine', 'asparagine', 'aspartic acid', 'cysteine', 'glutamic acid',
        'glutamine', 'glycine', 'histidine', 'isoleucine', 'leucine', 'lysine', 'methionine',
        'phenylalanine', 'proline', 'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine'
    ]
    for amino_acid in amino_acids:
        if amino_acid in text.lower():
            return amino_acid.capitalize()
    return ''

def handle_duplicates(ptm_data):
    primary_key_dict = defaultdict(list)

    for row in ptm_data:
        primary_key = tuple(row[:4])  # First four columns as primary key
        primary_key_dict[primary_key].append(row)

    deduplicated_data = []
    for rows in primary_key_dict.values():
        if len(rows) == 1:
            deduplicated_data.append(rows[0])
        else:
            # Mark all but the last occurrence as duplicate
            for row in rows[:-1]:
                new_row = row.copy()
                new_row[6] = "IGNORE"  # Set Status as "IGNORE"
                new_row[7] = "Duplicate"  # Set Notation as "Duplicate"
                deduplicated_data.append(new_row)
            # Keep the last occurrence with original status and notation
            deduplicated_data.append(rows[-1])

    return deduplicated_data

def determine_status(ptm_types, amino_acid_site, full_amino_acid, source, identifier, full_description):
    ptm_type_list = ptm_types.split(', ')
    aa_sites = amino_acid_site.split(', ') if amino_acid_site else []

    # Condition 1: IGNORE cases
    if (full_amino_acid and not amino_acid_site) or "Disulfide bond" in ptm_type_list:
        return "IGNORE", "N/A"

    # Condition 2: Incorrect format for Amino Acid and Site
    if not amino_acid_site:
        return "BAD", "N/A"

    # Condition 3: SKIP - Not matching PTM type and amino acid
    if len(ptm_type_list) == 1 and len(aa_sites) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if not validate_ptm_type(ptm_type_list[0], amino_acid):
            return "SKIP", "Not Matching"

    # Condition 4: Check source
    if source != "PTM":
        return "CHECK", "Source"

    # Condition 5: Single PTM type, single amino acid site, correct format, and matching amino acid
    if len(ptm_type_list) == 1 and len(aa_sites) == 1:
        amino_acid = aa_sites[0].split('-')[0]
        if validate_ptm_type(ptm_type_list[0], amino_acid):
            return "OKAY", "N/A"

    # Condition 6: SKIP - Alternate
    if 'alternate' in identifier.lower():
        return "SKIP", "Alternate"

    # Condition 7: MANUAL cases
    uncertainty_keywords = ["uncertain", "unknown", "unclear", "unsure", "not certain", "not known", "not clear", "not sure", "ambiguous", "undetermined"]
    if any(keyword in full_description.lower() for keyword in uncertainty_keywords):
        return "MANUAL", "Uncertainty"

    if len(aa_sites) > 1:
        site_numbers = [int(site.split('-')[1]) for site in aa_sites]
        site_numbers.sort()
        for i in range(len(site_numbers) - 1):
            if abs(site_numbers[i + 1] - site_numbers[i]) == 1:
                return "MANUAL", "Neighboring Sites"

    if len(ptm_type_list) > 1 and len(aa_sites) == 1:
        return "MANUAL", "Multi PTM and 1 Site"

    # Condition 8: CHECK - Multiple amino acid sites
    if len(aa_sites) > 1:
        if len(ptm_type_list) == 1:
            return "CHECK", "1 PTM and Multi Sites"
        else:
            return "CHECK", "Multi PTM and Multi Sites"

    # Default case
    return "CHECK", "Other"

def load_mod_mappings(file_path):
    mod_mappings = {}
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            identifier, feature_key, amino_acid, ptm_type, mod_id = row
            mod_mappings[identifier.split(';')[0].strip()] = mod_id
    return mod_mappings

def map_identifiers_to_mod(identifiers, mod_mappings):
    if identifiers == "N/A":
        return "N/A"

    mapped_mods = []
    for identifier in identifiers.split(" &&& "):
        base_identifier = identifier.split(';')[0].strip()
        if base_identifier in mod_mappings:
            mapped_mods.append(mod_mappings[base_identifier])
        else:
            mapped_mods.append("Not Found")

    return " &&& ".join(mapped_mods)


def process_ptm(first_accession, entry_name, ptm_info, ptm_data, source, ft_lines, mod_mappings):
    ptm_types = extract_all_ptm_types(ptm_info)
    amino_acid_site = extract_amino_acid_site(ptm_info)
    full_amino_acid = extract_amino_acid_fullname(ptm_info)

    if ptm_types and (amino_acid_site or full_amino_acid):
        identifier = extract_identifier(amino_acid_site, ft_lines) if amino_acid_site else "N/A"
        status, notation = determine_status(ptm_types, amino_acid_site, full_amino_acid, source, identifier, ptm_info)
        cross_reference = map_identifiers_to_mod(identifier, mod_mappings)

        ptm_data.append([
            first_accession,
            entry_name,
            ptm_types,
            amino_acid_site or full_amino_acid,
            source,
            ptm_info,
            status,
            notation,
            identifier,
            cross_reference
        ])

def extract_identifier(amino_acid_site, ft_lines):
    if not amino_acid_site:
        return "N/A"

    site_numbers = re.findall(r'\d+', amino_acid_site)
    identifiers = []

    for number in site_numbers:
        identifier = "N/A"
        for i, line in enumerate(ft_lines):
            if line.startswith(('FT   MOD_RES', 'FT   CARBOHYD', 'FT   LIPID')):  # Match with leading spaces
                line_parts = line.split()
                if len(line_parts) > 2 and line_parts[2] == number:
                    note_match = line[28:].strip().strip('"')
                    if note_match:
                        identifier = note_match.group(1)
                    else:
                        if i + 1 < len(ft_lines):
                            next_line = ft_lines[i + 1].strip()  # Strip leading spaces from the next line
                            if '/note=' in next_line:
                                note_match = re.search(r'/note="([^"]+)"', next_line)
                                if note_match:
                                    identifier = note_match.group(1)
                    break
        identifiers.append(identifier)

    return " &&& ".join(identifiers)

def validate_ptm_type(ptm_type, amino_acid):
    valid_combinations = {
    'Glycosylation': ['Gly', 'Arg', 'Asn', 'Ser', 'Thr', 'Tyr'],
    'Glutathionylation': ['Cys'],
    'Phosphorylation': ['Asp', 'Glu', 'Asn', 'Lys', 'Ser', 'Thr', 'Tyr', 'Arg', 'Cys', 'His'],
    'Ubiquitination': ['Lys'],
    'Acetylation': ['Lys', 'Arg', 'Ala', 'Asp', 'Cys', 'Glu', 'Gly', 'Ile', 'Met', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val'],
    'Methylation': ['Gln', 'Trp', 'Asp', 'Arg', 'Cys', 'Glu', 'Leu', 'Lys', 'His', 'Ala', 'Gly', 'Met', 'Ser', 'Pro', 'Asn', 'Ile', 'Phe', 'Tyr', 'Thr'],
    'Sumoylation': ['Lys'],
    'Hydroxylation': ['Arg', 'Asn', 'Asp', 'His', 'Pro', 'Ile', 'Leu', 'Tyr', 'Phe', 'Trp', 'Val', 'Lys', 'Glu', 'Ser', 'Cys'],
    'Nitration': ['Tyr'],
    'Prenylation': ['Trp', 'Cys'],
    'Bromination': ['Trp', 'His'],
    'ADP-ribosylation': ['Arg', 'Asn', 'Cys', 'His', 'Ser', 'Thr', 'Glu', 'Lys'],
    'Amidation': ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glu', 'Gln', 'Gly', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Pro', 'Thr', 'Trp', 'Tyr', 'Val', 'Phe'],
    'Citrullination': ['Arg'],
    'Oxidation': ['Cys', 'Met', 'Leu'],
    'Iodination': ['Tyr'],
    'Formylation': ['Gly', 'Met', 'Lys'],
    'S-nitrosylation': ['Cys'],
    'Sulfation': ['Cys', 'Ser', 'Thr', 'Tyr'],
    'Glycation': ['Lys']
}

    for key, value in valid_combinations.items():
        if key in ptm_type:
            return amino_acid in value
    return False

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession', 'Entry Name', 'PTM Type', 'Amino Acid and Site', 'Source', 'Full Description', 'Status', 'Notation', 'Identifier', 'Cross Reference'])
        for row in data:
            if row[2] and row[3]:  # Only write rows where both 'PTM Type' (index 2) and 'Amino Acid and Site' (index 3) are not empty
                writer.writerow(row)

def count_status_and_notation(data):
    status_counts = {'OKAY': 0, 'BAD': 0, 'CHECK': 0, 'IGNORE': 0, 'SKIP': 0, 'MANUAL': 0}
    notation_counts = {}

    for row in data:
        status = row[6]  # "Status" column is at index 6
        notation = row[7]  # "Notation" column is at index 7

        # Count statuses
        if status in status_counts:
            status_counts[status] += 1

        # Count notations
        if notation not in notation_counts:
            notation_counts[notation] = 0
        notation_counts[notation] += 1

    return status_counts, notation_counts

# Example usage
file_path_1 = '/content/drive/MyDrive/Protein Ontology-text mining/data/partial test 3.dat'
file_path_2 = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/PTM_list_PTMcombination_3.csv'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver_test.csv'

# Load MOD mappings from input file 2
mod_mappings = load_mod_mappings(file_path_2)

# Parse the UniProt file and extract PTM information
ptm_data = parse_uniprot_file(file_path_1, mod_mappings)

# Handle duplicates
deduplicated_ptm_data = handle_duplicates(ptm_data)

# Count the status and notations
status_counts, notation_counts = count_status_and_notation(deduplicated_ptm_data)

# Print the counts
print(f"Status Counts: {status_counts}")
print(f"Notation Counts: {notation_counts}")

# Write the extracted data to a CSV file
write_to_csv(deduplicated_ptm_data, output_file)

print(f"PTM summary has been written to {output_file}")


Status Counts: {'OKAY': 9, 'BAD': 0, 'CHECK': 1, 'IGNORE': 9, 'SKIP': 10, 'MANUAL': 3}
Notation Counts: {'Alternate': 10, 'N/A': 16, 'Duplicate': 2, 'Neighboring Sites': 2, 'Source': 1, 'Multi PTM and 1 Site': 1}
PTM summary has been written to /content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver_test.csv


In [ ]:
# Example usage
file_path = '/content/drive/MyDrive/Protein Ontology-text mining/data/partial test 3.dat'
output_file = '/content/drive/MyDrive/Protein Ontology-text mining/Filtering/ver_test.csv'